In [41]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from statsmodels.tsa.holtwinters import ExponentialSmoothing
import matplotlib.pyplot as plt

merged_df = pd.read_csv('merged_df.csv')
merged_df

,DATE,TEMP,TEMP_ATTRIBUTES,DEWP,DEWP_ATTRIBUTES,SLP,SLP_ATTRIBUTES,STP,STP_ATTRIBUTES,VISIB,...,MIN,MIN_ATTRIBUTES,PRCP,PRCP_ATTRIBUTES,SNDP,FRSHTT,YEAR,Month,Season,HUMID
0,2010-01-01,35.1,24,31.5,24,1013.0,21,12.7,24,6.2,...,30.2,,0.23,G,1.2,11000,2010,1,Winter,85.0
1,2010-01-02,29.2,24,20.5,24,1005.4,24,4.4,24,9.1,...,19.9,,0.01,G,NaN,11000,2010,1,Winter,66.0
2,2010-01-03,19.7,24,8.2,24,1001.1,22,999.9,24,8.9,...,17.1,,0.00,G,NaN,1000,2010,1,Winter,60.0
3,2010-01-04,24.2,24,11.6,24,1004.7,24,3.7,24,10.0,...,18.0,,0.00,G,NaN,0,2010,1,Winter,58.0
4,2010-01-05,25.4,24,12.8,24,1006.7,24,5.6,24,10.0,...,21.0,,0.00,G,NaN,0,2010,1,Winter,58.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5286,2024-09-26,68.2,24,62.3,24,1018.6,19,17.7,24,8.8,...,62.1,,0.01,G,NaN,10000,2024,9,Fall,84.0
5287,2024-09-27,70.0,24,62.5,24,1016.0,21,15.0,24,9.7,...,64.0,,0.07,G,NaN,10000,2024,9,Fall,73.0
5288,2024-09-28,66.2,24,59.4,24,1014.1,17,13.2,24,7.1,...,62.1,,0.05,G,NaN,10000,2024,9,Fall,84.0
5289,2024-09-29,62.1,24,57.0,24,1016.0,22,15.0,24,6.8,...,61.0,,0.22,G,NaN,10000,2024,9,Fall,82.0


In [42]:
merged_df = pd.read_csv('merged_df.csv')
merged_df['DATE'] = pd.to_datetime(merged_df['DATE'])
merged_df['FRSHTT'] = merged_df['FRSHTT'].astype(str).str.zfill(6)
merged_df['rain'] = merged_df['FRSHTT'].str[1].astype(int)

features = ['TEMP', 'HUMID', 'WDSP', 'VISIB']

# Build a daily DataFrame and add persistence lags
ts_daily = (
    merged_df
      .set_index('DATE')[features + ['rain']]
      .resample('D').mean()
      .ffill()
)

n_lags = 3
for lag in range(1, n_lags + 1):
    ts_daily[f'rain_lag{lag}'] = ts_daily['rain'].shift(lag).fillna(0)

feature_cols = features + [f'rain_lag{lag}' for lag in range(1, n_lags + 1)]

# Train/Test split
X = ts_daily[feature_cols]
y = ts_daily['rain']
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=0, stratify=y
)

# Fit Random Forest
rf = RandomForestClassifier(n_estimators=100, random_state=10)
rf.fit(X_train, y_train)
print(f"Test accuracy: {rf.score(X_test, y_test):.3f}")

# Compute climatology & blending weight
p_clim = ts_daily['rain'].mean()
w = 0.5
print(f"Climatology rain rate (p_clim): {p_clim:.3f}")

# Dynamic threshold for blended probabilities
test_probs_raw   = rf.predict_proba(X_test)[:, 1]
test_probs_blend = w * test_probs_raw + (1 - w) * p_clim
thresh_dyn       = np.quantile(test_probs_blend, 1 - p_clim)
print(f"Dynamic threshold (blended): {thresh_dyn:.3f}")

# Forecast meteorological features 7 days ahead
future_feats = {}
ts_feats = ts_daily[features]
for col in features:
    model = ExponentialSmoothing(
        ts_feats[col],
        trend='add',
        seasonal='add',
        seasonal_periods=365,
        initialization_method='estimated'
    )
    fit = model.fit()
    future_feats[col] = fit.forecast(7)

future_dates    = pd.date_range(ts_daily.index.max() + pd.Timedelta(days=1),
                                periods=7, freq='D')
future_X_feats  = pd.DataFrame(future_feats, index=future_dates)

# Iteratively predict with persistence, blending, and dynamic threshold
lag_vals = ts_daily[[f'rain_lag{lag}' for lag in range(1, n_lags + 1)]].iloc[-1].tolist()
results = []

for date, row in future_X_feats.iterrows():
    x_input   = np.array(list(row.values) + lag_vals).reshape(1, -1)
    raw_p     = rf.predict_proba(x_input)[:, 1][0]
    blend_p   = w * raw_p + (1 - w) * p_clim
    pred      = int(blend_p >= thresh_dyn)

    results.append({
        'DATE':            date,
        'rain_raw_prob':   raw_p,
        'rain_blend_prob': blend_p,
        'pred_rain':       pred
    })
    # update lag values for next iteration
    lag_vals = [pred] + lag_vals[:-1]

future_df = pd.DataFrame(results).set_index('DATE')
print(future_df)

output_path = 'rain_forecast.csv'
future_df.reset_index().to_csv(output_path, index=False)

Test accuracy: 0.814
Climatology rain rate (p_clim): 0.395
Dynamic threshold (blended): 0.407
            rain_raw_prob  rain_blend_prob  pred_rain
DATE                                                 
2024-10-01           0.94         0.667327          1
2024-10-02           0.95         0.672327          1
2024-10-03           0.96         0.677327          1
2024-10-04           0.91         0.652327          1
2024-10-05           0.77         0.582327          1
2024-10-06           0.88         0.637327          1
2024-10-07           0.91         0.652327          1
Rain forecast saved to rain_forecast.csv


C:\Users\Sara Kiel\PycharmProjects\IS525-Final-Project\.venv_tf\Lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
C:\Users\Sara Kiel\PycharmProjects\IS525-Final-Project\.venv_tf\Lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
C:\Users\Sara Kiel\PycharmProjects\IS525-Final-Project\.venv_tf\Lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
C:\Users\Sara Kiel\PycharmProjects\IS525-Final-Project\.venv_tf\Lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
C:\Users\Sara Kiel\PycharmProjects\IS525

In [43]:
import itertools

# verifying legitimacy of results - seems to predict a higher chance of rain than anticipated

daily_rain = (
    merged_df
      .set_index('DATE')['rain']
      .resample('D').max()
      .ffill()
)

# longest run of 1’s (rain) historically
runs = [sum(1 for _ in group)
        for val, group in itertools.groupby(daily_rain)
        if val == 1]
print("Longest rainy streak in history:", max(runs, default=0))

# historical rain frequency in October
oct_rain_rate = daily_rain[daily_rain.index.month==10].mean()
print("October rain probability:", oct_rain_rate)


Longest rainy streak in history: 12
October rain probability: 0.4055299539170507
